In [1]:

import argparse, json, math, os, sys, statistics
from typing import List, Dict, Tuple

In [5]:
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
# NLTK only for sentence splitting in feedback prompt shaping.
try:
    import nltk
    nltk.download("punkt", quiet=True)
except Exception:
    pass

In [6]:
def get_device():
    if torch.cuda.is_available():
        return "cuda"
    if torch.backends.mps.is_available():
        return "mps"
    return "cpu"

In [9]:

def load_phi2(model_name: str = "microsoft/phi-2",
              load_in_4bit: bool = True,
              device_map: str = "auto"):
    """
    Load Phi-2 with memory-friendly defaults. Falls back gracefully if bitsandbytes not available.
    """
    kwargs = {}
    if load_in_4bit:
        try:
            kwargs.update(dict(
                load_in_4bit=True,  # requires bitsandbytes
                device_map=device_map
            ))
        except Exception:
            # no bnb available; fall back to bf16 on GPU or float32 CPU
            pass
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Phi-2 sometimes needs pad token set for batching convenience
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            **kwargs
        )
    except Exception:
        # Fallback without 4bit
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map=device_map
        )

    # Ensure model uses pad_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    model.eval()
    return tokenizer, model


In [10]:
def sliding_window_perplexity(text: str,
                              tokenizer: AutoTokenizer,
                              model: AutoModelForCausalLM,
                              max_len: int = 2048,
                              stride: int = 1024) -> float:
    """
    Compute perplexity with sliding window for long texts.
    """
    enc = tokenizer(text, return_tensors="pt")
    input_ids = enc["input_ids"].to(model.device)

    nlls = []
    seq_len = input_ids.size(1)

    for i in range(0, seq_len, stride):
        begin = i
        end = min(i + max_len, seq_len)
        trg_len = end - i  # all tokens contribute to loss in this window

        input_ids_slice = input_ids[:, begin:end]
        with torch.no_grad():
            out = model(input_ids_slice, labels=input_ids_slice)
            # loss is mean over tokens in slice
            neg_log_likelihood = out.loss * trg_len
        nlls.append(neg_log_likelihood)

        if end == seq_len:
            break

    nll = torch.stack(nlls).sum()
    ppl = torch.exp(nll / seq_len)
    return float(ppl)


In [11]:
def load_dataset(path: str) -> Dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

In [12]:
def map_label(raw: str) -> int:
    """
    Map labels to binary target for AI detection:
    - AI / Hybrid => 1 (AI-like)
    - Human      => 0
    """
    raw = (raw or "").strip().lower()
    if raw in ("ai", "hybrid"):
        return 1
    return 0

In [13]:

def pick_threshold(perplexities: List[float], y_true: List[int]) -> Tuple[float, Dict[str, float]]:
    """
    Pick threshold that maximizes F1 on y_true by sweeping unique perplexity values.
    We predict AI if perplexity < threshold.
    """
    # Sort unique candidates; add small epsilon shifts
    candidates = sorted(set(perplexities))
    best = {"f1": -1.0}
    best_t = None

    for t in candidates:
        y_pred = [1 if p < t else 0 for p in perplexities]
        p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
        acc = accuracy_score(y_true, y_pred)
        if f1 > best["f1"]:
            best = {"precision": p, "recall": r, "f1": f1, "acc": acc}
            best_t = t
    return best_t, best

In [14]:
def generate_feedback(model, tokenizer, essay: str, rubric: Dict, max_new_tokens: int = 220) -> str:
    """
    Simple rubric-aligned feedback prompt for Phi-2 (base model).
    """
    criteria_lines = []
    for c in rubric.get("criteria", []):
        criteria_lines.append(f"- {c['name']}: {c['description']}")

    prompt = (
        "You are an academic writing assistant. Read the student's essay and provide constructive, rubric-aligned feedback. "
        "Focus on strengths, specific areas to improve, and one actionable suggestion per criterion. Be concise and professional.\n\n"
        f"RUBRIC ({rubric.get('rubric_id','')} / {rubric.get('domain','')}):\n"
        + "\n".join(criteria_lines) +
        "\n\nSTUDENT ESSAY:\n" + essay.strip() + "\n\nFEEDBACK:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # Return only the part after "FEEDBACK:" to keep it clean
    return text.split("FEEDBACK:", 1)[-1].strip()

In [45]:
def analyze_essay_and_generate_feedback(essay: str,
                                        tokenizer,
                                        model,
                                        rubric: Dict,
                                        calibration_perplexities: List[float],
                                        calibration_y_true: List[int]):
    """
    Analyzes an essay for potential AI generation using perplexity and
    generates feedback if classified as AI.

    Args:
        essay: The text of the essay to analyze.
        tokenizer: The loaded tokenizer.
        model: The loaded language model.
        rubric: The rubric dictionary.
        calibration_perplexities: List of perplexity scores from the calibration data.
        calibration_y_true: List of true labels (0 for Human, 1 for AI/Hybrid)
                            for the calibration data.

    Returns:
        A tuple containing:
        - prediction_label (str): The AI detection classification ("Human" or "AI/Hybrid").
        - feedback (str): The generated feedback if classified as AI, or a message
                          indicating no feedback was generated if classified as Human.
    """
    # 1. Calculate perplexity for the input essay
    essay_perplexity = sliding_window_perplexity(essay, tokenizer, model, max_len=2048, stride=1024)
    print(f"[INFO] Input essay perplexity: {essay_perplexity:.2f}")

    # 2. Calibrate the threshold using the provided calibration data
    # Note: In a real application, you might load/save the threshold instead
    # of recalibrating every time if the calibration data is large.
    if not calibration_perplexities or not calibration_y_true:
         print("[WARN] Calibration data is empty. Cannot perform AI detection.")
         return "Unknown", "Cannot perform AI detection due to missing calibration data."

    threshold, scores = pick_threshold(calibration_perplexities, calibration_y_true)
    print(f"[INFO] Calibrated AI detection threshold: {threshold:.2f}")

    # 3. Classify the essay based on the calibrated threshold
    prediction = 1 if essay_perplexity < threshold else 0
    prediction_label = "AI/Hybrid" if prediction == 1 else "Human"
    print(f"[INFO] Essay classified as: {prediction_label}")

    # 4. Generate feedback if classified as AI
    feedback = ""
    if prediction == 1:
        print("[INFO] Generating feedback...")
        feedback = generate_feedback(model, tokenizer, essay, rubric, max_new_tokens=220)
    else:
        feedback = "Essay classified as Human. No feedback generated based on AI detection."

    return prediction_label, feedback

# --- Example Usage ---

# Ensure model, tokenizer, data, rubric, submission_perplexities, and submission_y_true are loaded
# You might need to run the previous cells to load these variables.

if 'model' in locals() and 'tokenizer' in locals() and 'data' in locals() and 'submission_perplexities' in locals() and 'submission_y_true' in locals():

    # Define a sample essay to test the function
    test_essay = """
    Artificial intelligence is rapidly transforming various industries.
    Its impact on healthcare is particularly noteworthy, enabling faster and more accurate diagnoses.
    AI-powered tools are also revolutionizing the financial sector through algorithmic trading and fraud detection.
    The potential benefits are immense, but ethical considerations regarding job displacement and bias in algorithms must be addressed.
    Governments and organizations need to collaborate to ensure responsible development and deployment of AI technologies.
    """

    print("\n--- Analyzing Test Essay ---")
    detection_result, generated_feedback = analyze_essay_and_generate_feedback(
        test_essay,
        tokenizer,
        model,
        rubric,
        submission_perplexities,
        submission_y_true
    )

    print(f"\nAI Detection Result: {detection_result}")
    print(f"\nGenerated Feedback:\n{generated_feedback}")

else:
    print("Please run the necessary cells to load the model, tokenizer, data, and calibration data before running this cell.")


--- Analyzing Test Essay ---
[INFO] Input essay perplexity: 4.51
[INFO] Calibrated AI detection threshold: 12.99
[INFO] Essay classified as: AI/Hybrid
[INFO] Generating feedback...

AI Detection Result: AI/Hybrid

Generated Feedback:
Theoretical Understanding: The student demonstrates a basic understanding of the topic by acknowledging the transformative nature of AI in different industries.
Critical Analysis: The student provides a general overview of the impact of AI in healthcare and finance but lacks in-depth analysis or evaluation of psychological concepts or evidence.
Evidence Integration: The student includes some examples of AI-powered tools in healthcare and finance but does not effectively integrate credible psychological research to support arguments.
Academic Writing: The student's writing quality is satisfactory, but there is room for improvement in organization and use of APA style. The use of psychological terminology is appropriate.

1. Provide specific suggestions for